In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import BayesianGaussianMixture
from sklearn.cluster import KMeans
from scipy.special import digamma, logsumexp
from utils import plot_confidence_ellipse


In [ ]:
X = np.loadtxt('data/gaussian.txt')
X = (X - X.mean(axis=0)) / X.std(axis=0)
N, D = X.shape  # n_samples, n_features

In [ ]:
m_true = np.array([[0, 0], [3, -3], [3, 3], [-3, 3], [-3, -3]])
covs_true = np.array([[[1, 0], [0, 1]], [[1, 0.5], [0.5, 1]], [[1, -0.5], [-0.5, 1]], [[1, 0.5], [0.5, 1]], [[1, -0.5], [-0.5, 1]]])
X = np.concatenate([np.random.multivariate_normal(m_true[k], covs_true[k], 100) for k in range(len(m_true))])
N, D = X.shape

plt.figure(figsize=(5,5))
ax = plt.gca()
for k in range(len(m_true)):
    plot_confidence_ellipse(m_true[k], covs_true[k], 0.9, ax=ax, ec='teal')
plt.plot(*X.T, '.')
ax.set_aspect('equal')

In [ ]:
def init():
#     global invW0, invW, resp, m, m0, beta0, invS0, invS, v0, v, pi
    global m0, invW0, beta0, v0, alpha0, resp
    m0 = X.sum(axis=0)
#     m = X[np.random.choice(N, K)]
    invW0 = np.cov(X.T)
#     W = np.array([np.cov(X[np.random.choice(N, 10)].T) for _ in range(K)])
#     invW = np.linalg.inv(W)
    beta0 = 1.
#     invS0 = np.linalg.inv(beta0 * np.eye(D))
#     invS = np.array([invS0 for _ in range(K)])
    v0 = D
#     v = v0 * np.ones(K)
#     pi = np.ones(K) / K
    alpha0 = 1 / K
    
    resp = np.random.rand(N, K)
    resp /= resp.sum(axis=1)[:, np.newaxis]
    
    resp = np.zeros((N, K))
    label = KMeans(n_clusters= K, n_init=1).fit(X).labels_
    resp[np.arange(N), label] = 1
    resp /= resp.sum(axis=1)[:, np.newaxis]
    resp = np.loadtxt('data/resp.txt')
    

In [ ]:
def m_step():
    global pi
    pi = resp.sum(axis=0) / resp.sum()

In [ ]:
def compute_esp():
    global esp_T, esp_log_det_T, esp_mu#, esp_mu_muT
    global esp_log_pi
    W = np.linalg.inv(invW)
    esp_T = np.zeros_like(W)
    esp_log_det_T = np.zeros(K)
    esp_mu = np.copy(m)
    esp_log_pi = digamma(alpha) - digamma(alpha.sum())
#     esp_mu_muT = np.zeros_like(invS)
    for k in range(K):
        esp_T[k] = v[k] * W[k]
        esp_log_det_T[k] = digamma(0.5*(v[k] - np.arange(D))).sum() + D * np.log(2) - np.log(np.linalg.det(invW[k]))
#         esp_mu_muT[k] = invS[k] + np.outer(m[k], m[k])

In [ ]:
def compute_resp():
    global resp
    log_rho = np.zeros((N, K))
    for n in range(N):
        for k in range(K):
            log_rho[n, k] = esp_log_pi[k] + 0.5 * esp_log_det_T[k] - 0.5 * np.trace(
                esp_T[k] @ np.outer(X[n] - esp_mu[k], X[n] - esp_mu[k])) - 0.5 * D / beta[k] 
    log_resp = log_rho - logsumexp(log_rho, axis=1)[:, np.newaxis]
    resp = np.exp(log_resp)

In [ ]:
def update_params():
    global m, v, invW, alpha, beta
    invW = np.zeros((K, D, D))
    
    eta = resp.sum(axis=0) + 10*np.finfo(resp.dtype).eps
    v = v0 + eta
    alpha = alpha0 + eta
    beta = beta0 + eta
    m = (beta0 * m0 + resp.T @ X) / beta[:,np.newaxis]

    for k in range(K):
        s = np.zeros((D, D))
        for n in range(N):
            s += resp[n, k] * (np.outer(X[n], X[n]))
        invW[k] = invW0 + s + beta0 * np.outer(m0, m0) - beta[k] * np.outer(m[k], m[k])

In [ ]:
def display():
    plt.figure(figsize=(6,6))
    plt.plot(*X.T, 'o', c='dimgrey', alpha = 0.5)
    ax = plt.gca()
    
    weights = alpha / alpha.sum()
    for k in range(K):
        if weights[k] >= 1/(2*K):
            plot_confidence_ellipse(m[k], invW[k]/v[k], 0.9, ax=ax, ec='teal')
#     ax.set_aspect('equal')
    plt.show()

In [ ]:
K = 10
init()
update_params()
display()

for i in range(200):
    compute_esp()
    compute_resp()
    update_params()
#     m_step()
    if i % 20 == 0:
        print(i)
        display()